In [2]:
import subprocess
import sys
import os
import json

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install('kaggle')
install("pandas")
install("numpy")
install("matplotlib")
install("seaborn")


 Criando o arquivo `kaggle.json` para autenticação na plataforma da Kaggle

In [3]:
install("python-dotenv")

from dotenv import load_dotenv

load_dotenv()

kaggle_config = {
    "username": os.getenv('USER'),
    "key": os.getenv('KEY')
}

os.makedirs(os.path.join(os.path.expanduser("~"), ".kaggle"), exist_ok=True)

with open(os.path.join(os.path.expanduser("~"), ".kaggle/kaggle.json"), "w") as file:
    json.dump(kaggle_config, file)

os.chmod(os.path.join(os.path.expanduser("~"), ".kaggle/kaggle.json"), 0o600)

Baixando o dataset `brazilian-ecommerce`

In [16]:
def download_kaggle_dataset(dataset):
    subprocess.run(["kaggle", "datasets", "download", "-d", dataset])

download_kaggle_dataset("olistbr/brazilian-ecommerce")

Dezipando o arquivo do dataset

In [5]:

import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


with zipfile.ZipFile("brazilian-ecommerce.zip", "r") as zip_ref:
    zip_ref.extractall("brazilian_ecommerce")


Acessando o dataset através da biblioteca `pandas`

In [17]:
# Criação de uma lista para identificar valores ausentes
lista_labels_valores_ausentes = ["n/a", "na", "undefined"]

# Carregamento dos datasets
customers = pd.read_csv("brazilian_ecommerce/olist_customers_dataset.csv", na_values = lista_labels_valores_ausentes)
geolocation = pd.read_csv("brazilian_ecommerce/olist_geolocation_dataset.csv", na_values = lista_labels_valores_ausentes)
order_items = pd.read_csv("brazilian_ecommerce/olist_order_items_dataset.csv", na_values = lista_labels_valores_ausentes)
payments = pd.read_csv("brazilian_ecommerce/olist_order_payments_dataset.csv", na_values = lista_labels_valores_ausentes)
reviews = pd.read_csv("brazilian_ecommerce/olist_order_reviews_dataset.csv", na_values = lista_labels_valores_ausentes)
orders = pd.read_csv("brazilian_ecommerce/olist_orders_dataset.csv", na_values = lista_labels_valores_ausentes)
products = pd.read_csv("brazilian_ecommerce/olist_products_dataset.csv", na_values = lista_labels_valores_ausentes)
sellers = pd.read_csv("brazilian_ecommerce/olist_sellers_dataset.csv", na_values = lista_labels_valores_ausentes)
category = pd.read_csv("brazilian_ecommerce/product_category_name_translation.csv", na_values = lista_labels_valores_ausentes)


Unindo todas as tabelas

In [18]:
# Unindo as tabelas
merged_data = pd.merge(orders, customers, on='customer_id')
merged_data = pd.merge(merged_data, order_items, on='order_id')
merged_data = pd.merge(merged_data, products, on='product_id')
merged_data = pd.merge(merged_data, reviews, on='order_id')

Mostrando o total de registros por dataset

In [19]:
# Total de registros por dataset
count_array = np.array([
    ["customers", len(customers)],
    ["geolocation", len(geolocation)],
    ['geolocation', len(geolocation)],
    ['order_items', len(order_items)],
    ['payments', len(payments)],
    ['reviews', len(reviews)],
    ['orders', len(orders)],
    ['products', len(products)],
    ['sellers', len(sellers)],
    ['category', len(category)]
])

count_record = pd.DataFrame(count_array, columns = ["Tabela", "Total de Registros"])
print(count_record)

        Tabela Total de Registros
0    customers              99441
1  geolocation            1000163
2  geolocation            1000163
3  order_items             112650
4     payments             103886
5      reviews              99224
6       orders              99441
7     products              32951
8      sellers               3095
9     category                 71


Criando uma função para fazer um resumo da tabela informada

In [10]:
# Função para resumir a tabela informada
def resumo_tabela(tabela):
    print(f"Quantidade de colunas: {tabela.shape[1]}\nQuantidade de registros: {tabela.shape[0]}\n")

    porcentagem_nulls = (tabela.isnull().sum() / tabela.shape[0]) * 100

    print("Porcentagem de valores ausentes por coluna:")
    print(round(porcentagem_nulls.sort_values(ascending=False), 2))

    print(tabela.head(2))


Mostrando o resumo as Tabelas

In [11]:
resumo_tabela(customers)

Quantidade de colunas: 5
Quantidade de registros: 99441

Porcentagem de valores ausentes por coluna:
customer_id                 0.0
customer_unique_id          0.0
customer_zip_code_prefix    0.0
customer_city               0.0
customer_state              0.0
dtype: float64
                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   

   customer_zip_code_prefix          customer_city customer_state  
0                     14409                 franca             SP  
1                      9790  sao bernardo do campo             SP  


In [12]:
resumo_tabela(geolocation)

Quantidade de colunas: 5
Quantidade de registros: 1000163

Porcentagem de valores ausentes por coluna:
geolocation_zip_code_prefix    0.0
geolocation_lat                0.0
geolocation_lng                0.0
geolocation_city               0.0
geolocation_state              0.0
dtype: float64
   geolocation_zip_code_prefix  geolocation_lat  geolocation_lng  \
0                         1037       -23.545621       -46.639292   
1                         1046       -23.546081       -46.644820   

  geolocation_city geolocation_state  
0        sao paulo                SP  
1        sao paulo                SP  


In [13]:
resumo_tabela(order_items)

Quantidade de colunas: 7
Quantidade de registros: 112650

Porcentagem de valores ausentes por coluna:
order_id               0.0
order_item_id          0.0
product_id             0.0
seller_id              0.0
shipping_limit_date    0.0
price                  0.0
freight_value          0.0
dtype: float64
                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1   

                         product_id                         seller_id  \
0  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
1  e5f2d52b802189ee658865ca93d83a8f  dd7ddc04e1b6c2c614352b383efe2d36   

   shipping_limit_date  price  freight_value  
0  2017-09-19 09:45:35   58.9          13.29  
1  2017-05-03 11:05:13  239.9          19.93  


In [ ]:
resumo_tabela(payments)

Quantidade de colunas: 5
Quantidade de registros: 103886

Porcentagem de valores ausentes por coluna:
order_id                0.0
payment_sequential      0.0
payment_type            0.0
payment_installments    0.0
payment_value           0.0
dtype: float64
                           order_id  payment_sequential payment_type  \
0  b81ef226f3fe1789b1e8b2acac839d17                   1  credit_card   
1  a9810da82917af2d9aefd1278f1dcfa0                   1  credit_card   

   payment_installments  payment_value  
0                     8          99.33  
1                     1          24.39  


In [ ]:
resumo_tabela(reviews)

Quantidade de colunas: 7
Quantidade de registros: 99224

Porcentagem de valores ausentes por coluna:
review_comment_title       88.34
review_comment_message     58.70
review_id                   0.00
order_id                    0.00
review_score                0.00
review_creation_date        0.00
review_answer_timestamp     0.00
dtype: float64
                          review_id                          order_id  \
0  7bc2406110b926393aa56f80a40eba40  73fc7af87114b39712e6da79b0a377eb   
1  80e641a11e56f04c1ad469d5645fdfde  a548910a1c6147796b98fdf73dbeba33   

   review_score review_comment_title review_comment_message  \
0             4                  NaN                    NaN   
1             5                  NaN                    NaN   

  review_creation_date review_answer_timestamp  
0  2018-01-18 00:00:00     2018-01-18 21:46:59  
1  2018-03-10 00:00:00     2018-03-11 03:05:13  


In [ ]:
resumo_tabela(orders)

Quantidade de colunas: 8
Quantidade de registros: 99441

Porcentagem de valores ausentes por coluna:
order_delivered_customer_date    2.98
order_delivered_carrier_date     1.79
order_approved_at                0.16
order_id                         0.00
customer_id                      0.00
order_status                     0.00
order_purchase_timestamp         0.00
order_estimated_delivery_date    0.00
dtype: float64
                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2018-07-26 14:31:00         

In [ ]:
resumo_tabela(products)

Quantidade de colunas: 9
Quantidade de registros: 32951

Porcentagem de valores ausentes por coluna:
product_category_name         1.85
product_name_lenght           1.85
product_description_lenght    1.85
product_photos_qty            1.85
product_weight_g              0.01
product_length_cm             0.01
product_height_cm             0.01
product_width_cm              0.01
product_id                    0.00
dtype: float64
                         product_id product_category_name  \
0  1e9e8ef04dbcff4541ed26657ea517e5            perfumaria   
1  3aa071139cb16b67ca9e5dea641aaa2f                 artes   

   product_name_lenght  product_description_lenght  product_photos_qty  \
0                 40.0                       287.0                 1.0   
1                 44.0                       276.0                 1.0   

   product_weight_g  product_length_cm  product_height_cm  product_width_cm  
0             225.0               16.0               10.0              14.0  
1     

In [ ]:
resumo_tabela(sellers)

Quantidade de colunas: 4
Quantidade de registros: 3095

Porcentagem de valores ausentes por coluna:
seller_id                 0.0
seller_zip_code_prefix    0.0
seller_city               0.0
seller_state              0.0
dtype: float64
                          seller_id  seller_zip_code_prefix seller_city  \
0  3442f8959a84dea7ee197c632cb2df15                   13023    campinas   
1  d1b65fc7debc3361ea86b5f14c68d2e2                   13844  mogi guacu   

  seller_state  
0           SP  
1           SP  


In [ ]:
resumo_tabela(category)

Quantidade de colunas: 2
Quantidade de registros: 71

Porcentagem de valores ausentes por coluna:
product_category_name            0.0
product_category_name_english    0.0
dtype: float64
    product_category_name product_category_name_english
0            beleza_saude                 health_beauty
1  informatica_acessorios         computers_accessories


Contagem de valores únicos por categoria

In [ ]:
products["product_category_name"].value_counts()

product_category_name
cama_mesa_banho                  3029
esporte_lazer                    2867
moveis_decoracao                 2657
beleza_saude                     2444
utilidades_domesticas            2335
                                 ... 
fashion_roupa_infanto_juvenil       5
casa_conforto_2                     5
pc_gamer                            3
seguros_e_servicos                  2
cds_dvds_musicais                   1
Name: count, Length: 73, dtype: int64

In [ ]:
produtoMaisCaro = merged_data.loc[merged_data["price"].idxmax()]

produtoMaisCaro[['product_id', 'product_category_name', 'price']]

product_id               489ae2aa008f021502940f251d4cce7f
product_category_name               utilidades_domesticas
price                                              6735.0
Name: 79443, dtype: object

In [ ]:
produtoMaisBarato = merged_data.loc[merged_data["price"].idxmin()]

produtoMaisBarato[["product_id", "product_category_name", "price"]]

product_id                8a3254bee785a526d548a81a9bc3c9be
product_category_name    construcao_ferramentas_construcao
price                                                 0.85
Name: 79236, dtype: object